# Intialisation Steps
Connecting to Snowflake

## Importing required modules

In [2]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [3]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Queries from here

# FTD and PSP

## Relevant ICD codes

In [4]:
snow.select("select * from RWD_DB.RWD.ICD_GROUPER limit 3")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,1,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A00,No map,No map,Cholera,Cholera,2017-11-02,2017-11-02
1,2,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A000,0010,Cholera due to vibrio cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,2017-11-02,2017-11-02
2,3,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A001,0011,Cholera due to vibrio cholerae el tor,Cholera due to Vibrio cholerae 01 biovar eltor,Cholera due to Vibrio cholerae 01 biovar eltor,2017-11-02,2017-11-02


In [8]:
%%read_sql
select 
  * 
from
    RWD_DB.RWD.ICD_GROUPER

where    
    ((level_4_long_description_icd10 ilike '%frontotemporal%'
    AND level_4_long_description_icd10 	ilike '%dementia%')
     OR (level_4_long_description_icd10 ilike '%progressive%'
        AND level_4_long_description_icd10 ilike '%supranuclear%'))
        --AND level_4_long_description_icd10 ilike '%palsy%'

Query started at 04:00:17 PM India Standard Time; Query executed in 0.06 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,5862,G00 - G998,Diseases of the nervous system,G20-G26,Extrapyramidal and movement disorders,G23-G239,Other degenerative diseases of basal ganglia,G231,3330,Other degenerative diseases of the basal ganglia,Progressive supranuclear ophthalmoplegia,Progressive supranuclear ophthalmoplegia [Stee...,2017-11-02,2017-11-02
1,5905,G00 - G998,Diseases of the nervous system,G30-G32,Other degenerative diseases of the nervous system,G31-G319,Oth degenerative diseases of nervous system NEC,G310,No map,No map,Frontotemporal dementia,Frontotemporal dementia,2017-11-02,2017-11-02
2,5907,G00 - G998,Diseases of the nervous system,G30-G32,Other degenerative diseases of the nervous system,G31-G319,Oth degenerative diseases of nervous system NEC,G3109,33119,Other frontotemporal dementia,Other frontotemporal dementia,Other frontotemporal dementia,2017-11-02,2017-11-02


# Patient counts

## FTD patients
ICD codes: 
    level_4 - G310
    level_4 - G3109
    icd9_mapped_codes - 33119

In [9]:
%%read_sql
create or replace table st_ftd as
select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in ('G310', 'G3109', '33119')

Query started at 04:25:02 PM India Standard Time; Query executed in 15.58 m

,status
0,Table ST_FTD successfully created.


In [10]:
snow.select ("select count (distinct patient_id) from st_ftd where left(patient_id, 5) != 'XXX -'")

,COUNT (DISTINCT PATIENT_ID)
0,69322


## PSP patients
ICD codes: G231, 3330

In [11]:
%%read_sql
create or replace table st_psp as
select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in ('G231', '3330')

Query started at 05:17:34 PM India Standard Time; Query executed in 10.13 m

,status
0,Table ST_PSP successfully created.


In [12]:
snow.select ("select count (distinct patient_id) from st_psp where left(patient_id, 5) != 'XXX -'")

,COUNT (DISTINCT PATIENT_ID)
0,61350


## FTD and PSP patients

In [22]:
%%read_sql
create or replace table st_ftd_psp as
    select a.*,
           b.diagnosis as diag2
from 
    st_ftd a
inner join 
    st_psp b
on a.claim_number=b.claim_number

Query started at 06:34:26 PM India Standard Time; Query executed in 0.09 m

,status
0,Table ST_FTD_PSP successfully created.


In [23]:
snow.select ("select * from ST_ftd_psp limit 5")

,patient_id,claim_number,diagnosis,diagnosis_sequence,year_of_service,diag2
0,9sCCfYwL8Ri32nFJ,vul_40dfdf5d-e44d-40af-990b-2963cf032bc8,G3109,3,2016-08-09,G231
1,9sCCfYwL8Ri32nFJ,vul_40dfdf5d-e44d-40af-990b-2963cf032bc8,G3109,3,2016-08-09,G231
2,822Umhj5X+TTOkAy,vul_2aa303e3-f5fd-4738-883e-91aeff1ba8ba,G3109,2,2016-08-03,G231
3,9sCCfYwL8Ri32nFJ,vul_110848542,G3109,3,2016-08-09,G231
4,9sCCfYwL8Ri32nFJ,vul_110848542,G3109,3,2016-08-09,G231


In [24]:
snow.select ("select count (distinct patient_id) from st_ftd_psp")

,COUNT (DISTINCT PATIENT_ID)
0,447


# Nish from here

In [10]:
%%read_sql a

-- number of FTD patients
select
    count(distinct patient_id)
from
    st_ftd
where 
    left(patient_id, 5) != 'XXX -'

Query started at 02:27:34 PM GMT Daylight Time; Query executed in 0.04 m

,COUNT(DISTINCT PATIENT_ID)
0,69322


In [11]:
%%read_sql b

-- number of PSP patients
select
    count(distinct patient_id)
from
    st_psp
where 
    left(patient_id, 5) != 'XXX -'

Query started at 02:27:37 PM GMT Daylight Time; Query executed in 0.04 m

,COUNT(DISTINCT PATIENT_ID)
0,61350


In [12]:
%%read_sql c

-- number of FTD patients who also have PSP codes
select
    count(distinct patient_id)
from
    st_ftd
where 
    left(patient_id, 5) != 'XXX -'
    and patient_id in (select distinct patient_id
                       from st_psp)

Query started at 02:27:41 PM GMT Daylight Time; Query executed in 0.03 m

,COUNT(DISTINCT PATIENT_ID)
0,1011


## Summary calculations

In [89]:
lst = ['ftd',
       'psp',
       'ftd_psp',
       'overlap_psp_in_ftd (%)']

d = round(c/a*100, 2)

x = pd.concat([a,b,c,d], axis=1)
x.columns = lst
x

,ftd,psp,ftd_psp,overlap_psp_in_ftd (%)
0,69322,61350,1011,1.46


# Try EHR

## FTD counts

In [91]:
%%read_sql

create or replace table nk_albatross_ftd as

select 
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    ICD9,
    ICD10
from 
    RWD_DB.RWD.ALBATROSS_EHR_problems 
where 
    upper(ICD10) in ('G310', 'G3109')
    or ICD9 = '33119'

Query started at 03:42:58 PM GMT Daylight Time; Query executed in 2.39 m

,status
0,Table NK_ALBATROSS_FTD successfully created.


In [94]:
snow.select("select count(distinct patient_id) from nk_albatross_ftd where left(patient_id, 5) != 'XXX -'")

,COUNT(DISTINCT PATIENT_ID)
0,4490


## PSP counts

In [93]:
%%read_sql

create or replace table nk_albatross_psp as

select 
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    ICD9,
    ICD10
from 
    RWD_DB.RWD.ALBATROSS_EHR_problems 
where 
    upper(ICD10) = 'G231'
    or ICD9 = '3330'

Query started at 03:45:51 PM GMT Daylight Time; Query executed in 2.22 m

,status
0,Table NK_ALBATROSS_PSP successfully created.


In [95]:
snow.select("select count(distinct patient_id) from nk_albatross_psp where left(patient_id, 5) != 'XXX -'")

,COUNT(DISTINCT PATIENT_ID)
0,11157


In [96]:
snow.select("select count(distinct patient_id) from nk_albatross_ftd where left(patient_id, 5) != 'XXX -' and patient_id in (select distinct patient_id from nk_albatross_psp)")

,COUNT(DISTINCT PATIENT_ID)
0,113


In [97]:
snow.select("select count(distinct patient_id) from nk_albatross_psp where left(patient_id, 5) != 'XXX -' and patient_id in (select distinct patient_id from nk_albatross_ftd)")

,COUNT(DISTINCT PATIENT_ID)
0,113
